## Using CNN and NCM for CIFAR-10 feature extraction and classification

### Summary
CIFAR-10 데이터셋을 분류하기 위해 CNN을 이용하여 데이터셋에 대한 특징을 추출 후 NCM을 이용하여 각 class에 대한 평균 값을 이용하여 분류

<span style="color: #2D3748; background-color:#fff5b1;">Test size를 0.2로 10번 반복 실험한 결과 평균적으로 0.34의 정확도를 보여주고 있고, test 데이터 1개를 분류하는데 0.0011초의 시간이 걸린다.</span>

In [13]:
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from utils.util import Info
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from utils.Data_Classifier import train, validate

In [14]:
class Config(Info):
    def __init__(self):
        super(Info, self).__init__()
        self.device = 'PC'
        self.dataset = 'CIFAR_10'
        self.test_size = 0.2
        self.feature_size = 3072
        self.method = 'CNN'
        self.iter = 10

In [15]:
# cig = Config()
# cig.info()
# cig.print_rutin()

In [16]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

## Load CIFAR-10 Dataset

In [17]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                ])

batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

validationset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
val_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=2)

print(trainset.data.shape)
print(validationset.data.shape)

Files already downloaded and verified
Files already downloaded and verified
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

epoch = 100
num_class = 10

model = resnet18(num_classes=num_class)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [19]:
best_acc = 0.

for idx in range(epoch):  # loop over the dataset multiple times
    # train for one epoch
    train(train_loader, model, criterion, optimizer, idx, device)

    # evaluate on validation set
    acc = validate(val_loader, model, criterion, device)

    # remember best acc@1 and save checkpoint
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)

    scheduler.step()

print('Finished Training')

Epoch: [0]
Train: [98/98]	Time 0.0484	Data 0.0225	Loss 2.1219	Acc 33.14
Test: [20/20]	Time 0.0944	Data 0.0852	Loss 16.4620	Acc 39.76
Epoch: [1]
Train: [98/98]	Time 0.0491	Data 0.0227	Loss 1.5159	Acc 47.11
Test: [20/20]	Time 0.0937	Data 0.0826	Loss 1.3771	Acc 51.05
Epoch: [2]
Train: [98/98]	Time 0.0501	Data 0.0232	Loss 1.3283	Acc 53.97
Test: [20/20]	Time 0.0961	Data 0.0838	Loss 1.2378	Acc 55.38
Epoch: [3]
Train: [98/98]	Time 0.0482	Data 0.0226	Loss 1.1447	Acc 60.39
Test: [20/20]	Time 0.0902	Data 0.0801	Loss 1.1518	Acc 59.36
Epoch: [4]
Train: [98/98]	Time 0.0488	Data 0.0227	Loss 1.0506	Acc 64.86
Test: [20/20]	Time 0.0920	Data 0.0817	Loss 11.4038	Acc 48.03
Epoch: [5]
Train: [98/98]	Time 0.0501	Data 0.0233	Loss 0.9206	Acc 69.23
Test: [20/20]	Time 0.0973	Data 0.0846	Loss 1.0033	Acc 66.22
Epoch: [6]
Train: [98/98]	Time 0.0502	Data 0.0233	Loss 0.7450	Acc 74.36
Test: [20/20]	Time 0.0933	Data 0.0801	Loss 0.9484	Acc 68.70
Epoch: [7]
Train: [98/98]	Time 0.0501	Data 0.0232	Loss 0.6725	Acc 76.86
Te

In [20]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                      ])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                     ])

batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

validationset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
val_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=2)

print(trainset.data.shape)
print(validationset.data.shape)

Files already downloaded and verified
Files already downloaded and verified
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [21]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

epoch = 100
num_class = 10

model = resnet18(num_classes=num_class)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [22]:
best_acc = 0.

for idx in range(epoch):  # loop over the dataset multiple times
    # train for one epoch
    train(train_loader, model, criterion, optimizer, idx, device)

    # evaluate on validation set
    acc = validate(val_loader, model, criterion, device)

    # remember best acc and save checkpoint
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)

    scheduler.step()

print('Finished Training')

Epoch: [0]
Train: [98/98]	Time 0.0492	Data 0.0238	Loss 2.2192	Acc 29.30
Test: [20/20]	Time 0.0890	Data 0.0793	Loss 1.6696	Acc 39.29
Epoch: [1]
Train: [98/98]	Time 0.0483	Data 0.0221	Loss 1.5844	Acc 42.19
Test: [20/20]	Time 0.0902	Data 0.0806	Loss 1.9801	Acc 48.06
Epoch: [2]
Train: [98/98]	Time 0.0479	Data 0.0244	Loss 1.4539	Acc 47.83
Test: [20/20]	Time 0.0917	Data 0.0813	Loss 1.2942	Acc 53.01
Epoch: [3]
Train: [98/98]	Time 0.0483	Data 0.0231	Loss 1.2803	Acc 54.36
Test: [20/20]	Time 0.0906	Data 0.0811	Loss 1.2948	Acc 53.27
Epoch: [4]
Train: [98/98]	Time 0.0484	Data 0.0240	Loss 1.1471	Acc 59.13
Test: [20/20]	Time 0.0893	Data 0.0797	Loss 1.0677	Acc 62.38
Epoch: [5]
Train: [98/98]	Time 0.0481	Data 0.0247	Loss 1.0396	Acc 63.20
Test: [20/20]	Time 0.0893	Data 0.0797	Loss 0.9929	Acc 65.09
Epoch: [6]
Train: [98/98]	Time 0.0487	Data 0.0232	Loss 0.9662	Acc 65.99
Test: [20/20]	Time 0.0888	Data 0.0792	Loss 1.0136	Acc 64.74
Epoch: [7]
Train: [98/98]	Time 0.0477	Data 0.0233	Loss 0.9022	Acc 68.11
Test